# Secop

## ¿Qué contratos del Estado se van a Cerrar?

Los contratos que requiere ejecutar el estado se publican en la portal de Colombia Compra Eficiente por medio de su plataforma SECOP II y los interesados en participar en la licitación, se registran y entran en el concurso de adjudicación.

Nuestro interés en este proyecto era predecir cuando un contrato adjudicado por el estado tenia la probabilidad más alta de cerrarse según las características de contrato como,  de que Departamento(Estado), el orden, la modalidad, el destino del gasto entre otros aspectos del relevantes de la contratación.

La diferencia entre un contrato cerrado y un contrato terminado es que un contrato cerrado es un contrato que ha sido ejecutado en su totalidad, liquidado y archivado, mientras que un contrato terminado es un contrato que ha sido ejecutado en su totalidad, pero que aún no ha sido liquidado o archivado.

**Un contrato cerrado es un contrato que ha finalizado y ya no está vigente y las partes no pueden realizar ninguna modificación al contrato y no pueden iniciar un proceso de reclamación de daños si alguna de las partes ha incumplido sus obligaciones contractuales**

## Datos

Los datos fuente fueron descargados del portal de Datos Abiertos del Gobierno de Colombia que están disponible a todo el público.

https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos/jbjy-vk9h

El archivo original pesa 3.62 Gb por lo tanto se tomo este archivo fue procesado inicialmente para descartar varias columnas de datos que no eran relevante para nuestra proyección y fue realizado en el notebook Filtrado_datos_secop generando un arhivo ideal (datos_filtradosv1.csv) para iniciar con el tratamiento de datos.

**Diccionario de datos**

https://www.datos.gov.co/api/views/jbjy-vk9h/files/839439f9-b3b9-4e53-a28d-ab82e752a1dc?download=true&filename=Diccionario%20de%20Datos%20Abiertos%202022%20Contratos%20Electronicos.pdf

In [1]:
import pandas as pd

In [2]:
ruta_archivo = r"../data/processed/datos_filtradosv1.csv"
# Cargamos los datos del CSV a un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_12390/493588633.py:3: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


In [3]:
# Seleccionamos las columnas que queremos mantener
df = df[['Departamento',
         'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Tipo de Contrato',
         'Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion',
         'Es Grupo', 'Es Pyme', 'Liquidación',
         'Obligación Ambiental',
         'Valor del Contrato', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado',
         'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos Propios'
         ]]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2689335 entries, 0 to 2689334
Data columns (total 31 columns):
 #   Column                                                              Dtype  
---  ------                                                              -----  
 0   Departamento                                                        object 
 1   Orden                                                               object 
 2   Sector                                                              object 
 3   Rama                                                                object 
 4   Entidad Centralizada                                                object 
 5   Estado Contrato                                                     object 
 6   Tipo de Contrato                                                    object 
 7   Modalidad de Contratacion                                           object 
 8   Fecha de Firma                                                      obje

#### Conversion de las fechas a DateTime

In [4]:
columnas_fecha = ['Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato',
                  'Fecha de Inicio de Ejecucion', 'Fecha de Fin de Ejecucion']

# Iterar sobre las columnas y convertirlas al tipo de dato 'date'
for columna in columnas_fecha:
    df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
    df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')

#### Filtrar por el estado del contrato

In [5]:
estados_contrato = ["terminado", "Cerrado", "cedido", "Prorrogado", "Suspendido"]

# Filtrar las filas del dataframe basándote en la columna "Estado Contrato"
df_filtradoNoEjecucion = df[df["Estado Contrato"].isin(estados_contrato)]
df_filtradoNoEjecucion

,Departamento,Orden,Sector,Rama,Entidad Centralizada,Estado Contrato,Tipo de Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,...,Valor Pendiente de Ejecucion,Saldo CDP,EsPostConflicto,Destino Gasto,Origen de los Recursos,Dias Adicionados,Género Representante Legal,Presupuesto General de la Nacion – PGN,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios
0,Santander,Territorial,Servicio Público,Ejecutivo,Centralizada,terminado,Prestación de servicios,Contratación directa,2021-08-17,2021-08-19,...,0.0,13750000,No,Inversión,Distribuido,0,Hombre,0,13750000,0
2,Atlántico,Nacional,Educación Nacional,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2020-02-07,2020-02-07,...,0.0,366080000,No,Inversión,Distribuido,0,No Definido,0,0,0
3,Distrito Capital de Bogotá,Nacional,Hacienda y Crédito Público,Ejecutivo,No Definido,Cerrado,Prestación de servicios,Contratación directa,2019-04-15,2019-02-20,...,1557298.0,66379827,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
9,Distrito Capital de Bogotá,Nacional,agricultura,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2021-01-20,2021-01-21,...,1744200.0,1034967000,No,Inversión,Distribuido,0,Mujer,22032000,0,0
10,Distrito Capital de Bogotá,Nacional,Educación Nacional,Corporación Autónoma,Descentralizada,terminado,Prestación de servicios,Contratación régimen especial,2021-08-05,2021-08-09,...,2640000.0,52800000,No,Funcionamiento,Recursos Propios,0,No Definido,0,0,49500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689323,Cundinamarca,Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,Cerrado,Prestación de servicios,Contratación directa,2018-08-21,2018-08-21,...,20279040.0,20279040,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
2689327,Huila,Nacional,defensa,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Mínima cuantía,2021-04-06,2021-04-14,...,21000000.0,32550000,No,Funcionamiento,Distribuido,0,No Definido,24500000,0,0
2689331,Distrito Capital de Bogotá,Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,terminado,DecreeLaw092/2017,Contratación régimen especial,2022-03-24,2022-03-29,...,6957491.0,58912135,No,Funcionamiento,Recursos Propios,0,Mujer,0,0,6957491
2689332,Atlántico,Nacional,Trabajo,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2023-01-26,2023-02-01,...,14080000.0,2201918000,No,Inversión,Distribuido,0,No Definido,20680000,0,0


#### Filtrar por año del contrato

In [142]:
df_filtradoAnio = df_filtradoNoEjecucion[(df_filtradoNoEjecucion['Fecha de Inicio del Contrato'].dt.year == 2022) & (df_filtradoNoEjecucion['Fecha de Inicio del Contrato'].dt.month > 2)]

In [123]:
# df_filtradoAnio = df_filtradoAnio.drop_duplicates()
# df_filtradoAnio.info()

In [143]:
df_filtradoAnio.shape

(90220, 31)

#### Generando Nuevas Columnas

In [144]:
df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'
df_filtradoAnio = df_filtradoAnio.drop('Sector', axis=1)

df_filtradoAnio["EsPrestacionServicios"] = df_filtradoAnio['Tipo de Contrato'] == 'Prestación de servicios'
df_filtradoAnio = df_filtradoAnio.drop('Tipo de Contrato', axis=1)

df_filtradoAnio["EsGrupo"] = df_filtradoAnio['Es Grupo'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Grupo', axis=1)

df_filtradoAnio["EsPyme"] = df_filtradoAnio['Es Pyme'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Pyme', axis=1)

df_filtradoAnio["EstaLiquidado"] = df_filtradoAnio['Liquidación'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Liquidación', axis=1)

df_filtradoAnio["EsObligacionAmbiental"] = df_filtradoAnio['Obligación Ambiental'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Obligación Ambiental', axis=1)


df_filtradoAnio["Es PostConflicto"] = df_filtradoAnio['EsPostConflicto'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('EsPostConflicto', axis=1)

df_filtradoAnio["EsRecursosPropios"] = df_filtradoAnio['Origen de los Recursos'] == 'Recursos Propios'
df_filtradoAnio = df_filtradoAnio.drop('Origen de los Recursos', axis=1)


df_filtradoAnio = df_filtradoAnio.drop_duplicates()
df_filtradoAnio.info()

df_filtradoAnio.head(15)

<class 'pandas.core.frame.DataFrame'>
Index: 80301 entries, 25 to 2689331
Data columns (total 31 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   Departamento                                                        80301 non-null  object        
 1   Orden                                                               80301 non-null  object        
 2   Rama                                                                80301 non-null  object        
 3   Entidad Centralizada                                                80301 non-null  object        
 4   Estado Contrato                                                     80301 non-null  object        
 5   Modalidad de Contratacion                                           80301 non-null  object        
 6   Fecha de Firma                                          

/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_12390/2106044307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'


,Departamento,Orden,Rama,Entidad Centralizada,Estado Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,...,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios,EsServicioPublico,EsPrestacionServicios,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios
25,Caldas,Territorial,Ejecutivo,Descentralizada,terminado,Contratación directa,2022-12-01,2022-12-01,2022-12-31,NaT,...,3000000,0,True,True,False,False,False,False,False,False
33,Nariño,Nacional,Ejecutivo,Descentralizada,terminado,Contratación régimen especial,2022-09-14,2022-10-15,2022-12-31,NaT,...,0,0,False,False,False,False,False,False,False,False
78,Distrito Capital de Bogotá,Territorial,Ejecutivo,Descentralizada,cedido,Contratación directa,2022-07-05,2022-07-18,2023-03-04,NaT,...,0,16500000,False,True,False,False,False,False,False,True
88,Santander,Territorial,Ejecutivo,No Definido,terminado,Contratación directa,2022-10-07,2022-10-10,2022-11-09,NaT,...,4426300,0,False,True,False,False,False,False,False,False
89,Distrito Capital de Bogotá,Nacional,Ejecutivo,Descentralizada,Cerrado,Contratación directa,2022-09-07,2022-09-08,2022-12-31,NaT,...,0,0,False,True,False,False,False,False,False,False
116,Atlántico,Territorial,Corporación Autónoma,Descentralizada,terminado,Contratación régimen especial (con ofertas),2022-07-28,2022-07-28,2022-07-29,NaT,...,0,0,False,False,False,False,False,False,False,False
117,Casanare,Territorial,Ejecutivo,Centralizada,terminado,Selección Abreviada de Menor Cuantía,2022-11-04,2022-11-18,2023-01-01,NaT,...,93092681,0,False,False,False,True,True,False,False,False
136,Cundinamarca,Territorial,Ejecutivo,Descentralizada,terminado,Contratación directa,2022-08-30,2022-08-30,2022-12-31,NaT,...,18553333,0,False,True,False,False,False,False,False,False
195,Valle del Cauca,Territorial,Ejecutivo,Descentralizada,Cerrado,Contratación directa,2022-10-07,2022-10-13,2022-12-31,NaT,...,4500000,0,False,True,False,False,False,False,False,False
236,Santander,Territorial,Ejecutivo,Descentralizada,terminado,Contratación directa,2022-12-02,2022-12-05,2022-12-23,NaT,...,2216667,0,True,True,False,False,False,False,False,False


#### Generando nuevo archivo procedado

In [145]:
df_filtradoAnio.to_csv('datos_filtrados_20230719.csv', index=False)

In [146]:
!ls

Secop.ipynb
Secop_pipeline.joblib
datos_filtrados_20230719.csv
secopFiltro2021muestra_profile.html
secopFiltro2021muestra_profile20230718.html
secopFiltro2021muestra_profile_20230719_1923.html


In [147]:
!mv "datos_filtrados_20230719.csv" "../data/processed/"

In [148]:
#muestra_aleatoria = df_filtradoAnio.sample(frac=0.4)
#muestra_aleatoria.shape

In [149]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(muestra_aleatoria, title="Pandas Profiling Report")
# profile.to_file("secopFiltro2021muestra_profile_20230719_1923.html")

#### Separar variable a predecir

In [150]:
# Aplicar one-hot encoding
df_one_hot = pd.get_dummies(df_filtradoAnio['Estado Contrato'])

# Combinar el DataFrame original con las nuevas columnas
df_final = pd.concat([df_filtradoAnio, df_one_hot], axis=1)

# Eliminar la columna original "Estado Contrato" si deseas
df_final = df_final.drop('Estado Contrato', axis=1)

df_final

,Departamento,Orden,Rama,Entidad Centralizada,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,Fecha de Fin de Ejecucion,...,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios,Cerrado,Suspendido,cedido,terminado
25,Caldas,Territorial,Ejecutivo,Descentralizada,Contratación directa,2022-12-01,2022-12-01,2022-12-31,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
33,Nariño,Nacional,Ejecutivo,Descentralizada,Contratación régimen especial,2022-09-14,2022-10-15,2022-12-31,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
78,Distrito Capital de Bogotá,Territorial,Ejecutivo,Descentralizada,Contratación directa,2022-07-05,2022-07-18,2023-03-04,NaT,NaT,...,False,False,False,False,False,True,False,False,True,False
88,Santander,Territorial,Ejecutivo,No Definido,Contratación directa,2022-10-07,2022-10-10,2022-11-09,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
89,Distrito Capital de Bogotá,Nacional,Ejecutivo,Descentralizada,Contratación directa,2022-09-07,2022-09-08,2022-12-31,NaT,NaT,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689046,Nariño,Territorial,Ejecutivo,Descentralizada,Contratación directa,2022-10-13,2022-10-14,2022-12-28,NaT,NaT,...,False,False,False,False,False,False,True,False,False,False
2689126,Santander,Territorial,Corporación Autónoma,Descentralizada,Contratación directa,2022-09-29,2022-09-30,2022-12-29,NaT,NaT,...,False,False,False,False,False,True,False,False,True,False
2689230,Caldas,Territorial,Ejecutivo,Centralizada,Contratación directa,2022-08-02,2022-08-03,2022-09-12,NaT,NaT,...,False,False,True,False,False,False,False,False,False,True
2689283,Valle del Cauca,Territorial,Ejecutivo,Centralizada,Contratación directa,2022-07-14,2022-07-15,2022-09-30,NaT,NaT,...,False,False,False,False,False,False,True,False,False,False


In [151]:
estado_contrato = df_final["Cerrado"].copy()
contratos_data = df_final.drop(["terminado","cedido","Suspendido","Cerrado"],axis=1)

#### Split dataset

In [152]:
# Calculate test and validation set size:
original_count = len(df_final)
training_size = 0.60 # 60% of records
test_size = (1 - training_size) / 2


training_count = int(original_count * training_size)
test_count = int(original_count * test_size)
validation_count = original_count - training_count - test_count

print(training_count, test_count, validation_count, original_count)

48180 16060 16061 80301


In [153]:
contratos_data = contratos_data[["Departamento","Orden","Entidad Centralizada","Modalidad de Contratacion","Destino Gasto","Género Representante Legal","EsServicioPublico","EsRecursosPropios","EsGrupo","EsPrestacionServicios","EsPyme","EstaLiquidado","EsObligacionAmbiental","Es PostConflicto","Dias Adicionados","Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"]]

In [154]:
contratos_data.head(10)

,Departamento,Orden,Entidad Centralizada,Modalidad de Contratacion,Destino Gasto,Género Representante Legal,EsServicioPublico,EsRecursosPropios,EsGrupo,EsPrestacionServicios,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,Dias Adicionados,Valor del Contrato,Valor Facturado,Valor Pendiente de Pago,Saldo CDP
25,Caldas,Territorial,Descentralizada,Contratación directa,Funcionamiento,Hombre,True,False,False,True,False,False,False,False,0,3000000,3000000,0,3000000
33,Nariño,Nacional,Descentralizada,Contratación régimen especial,Funcionamiento,Hombre,False,False,False,False,False,False,False,False,0,5254080,0,5254080,0
78,Distrito Capital de Bogotá,Territorial,Descentralizada,Contratación directa,Inversión,No Definido,False,True,False,True,False,False,False,False,0,16500000,16500000,1246667,16500000
88,Santander,Territorial,No Definido,Contratación directa,Inversión,Mujer,False,False,False,True,False,False,False,False,0,4426300,4426300,0,4426300
89,Distrito Capital de Bogotá,Nacional,Descentralizada,Contratación directa,Inversión,Mujer,False,False,False,True,False,False,False,False,0,23000000,0,23000000,23000000
116,Atlántico,Territorial,Descentralizada,Contratación régimen especial (con ofertas),Funcionamiento,Hombre,False,False,False,False,False,False,False,False,0,2307560,2307560,0,40000000
117,Casanare,Territorial,Centralizada,Selección Abreviada de Menor Cuantía,Inversión,Hombre,False,False,False,False,True,True,False,False,0,93092681,0,93092681,93474331
136,Cundinamarca,Territorial,Descentralizada,Contratación directa,Inversión,Hombre,False,False,False,True,False,False,False,False,0,18553333,0,18553333,23000000
195,Valle del Cauca,Territorial,Descentralizada,Contratación directa,Inversión,Hombre,False,False,False,True,False,False,False,False,0,4500000,0,4500000,4500000
236,Santander,Territorial,Descentralizada,Contratación directa,Inversión,Hombre,True,False,False,True,False,False,False,False,0,2216666,2216666,0,2216667


In [155]:
from sklearn.model_selection import train_test_split

train_x, rest_x, train_y, rest_y = train_test_split(contratos_data, estado_contrato, train_size=training_count)
test_x, validate_x, test_y, validate_y = train_test_split(rest_x, rest_y, train_size=test_count)

print(len(train_x), len(test_x), len(validate_x))

48180 16060 16061


In [156]:
train_x

,Departamento,Orden,Entidad Centralizada,Modalidad de Contratacion,Destino Gasto,Género Representante Legal,EsServicioPublico,EsRecursosPropios,EsGrupo,EsPrestacionServicios,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,Dias Adicionados,Valor del Contrato,Valor Facturado,Valor Pendiente de Pago,Saldo CDP
152336,Boyacá,Territorial,Descentralizada,Contratación directa,Inversión,No Definido,True,False,False,True,False,False,False,False,0,7742644,7233260,509384,8659536
231057,Casanare,Territorial,Centralizada,Contratación directa,Inversión,Mujer,True,False,False,True,False,True,False,False,0,14400000,0,14400000,14400000
1222983,Antioquia,Territorial,Descentralizada,Selección Abreviada de Menor Cuantía,Inversión,No Definido,True,False,False,False,True,False,False,False,0,338100157,0,338100157,340852213
1604566,Meta,Territorial,Centralizada,Contratación directa,Inversión,Hombre,False,False,False,True,False,False,False,False,0,5400000,5400000,0,5400000
903739,Santander,Territorial,Descentralizada,Contratación directa,Funcionamiento,Mujer,False,True,False,True,False,False,False,False,0,6000000,6000000,6000000,6000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550563,"San Andrés, Providencia y Santa Catalina",Territorial,Descentralizada,Contratación directa,Inversión,Mujer,False,False,False,True,False,False,False,False,0,9204384,0,9204384,9204384
945967,"San Andrés, Providencia y Santa Catalina",Territorial,Descentralizada,Contratación directa,Inversión,Hombre,False,False,False,True,False,False,False,False,0,15412445,0,15412445,15412445
354794,Córdoba,Territorial,Descentralizada,Contratación régimen especial,Funcionamiento,Mujer,False,True,False,True,False,False,False,False,0,20687917,19860400,827517,615672400
290052,Meta,Nacional,Descentralizada,Mínima cuantía,Funcionamiento,No Definido,False,False,False,False,True,False,False,False,0,5745150,5745150,0,9245384


In [157]:
train_x.shape

(48180, 19)

## Codificando Variables - Pipeline

In [158]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [159]:
one_hot_encoding = ColumnTransformer([
    (
        'one_hot_encode',
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        [
            "Orden",
            "Entidad Centralizada",
            "Modalidad de Contratacion",
            "Destino Gasto",
            "Género Representante Legal",
            "Departamento",
        ]
    )
])

In [160]:
binarizer = ColumnTransformer([
    (
        'binarizer',
        Binarizer(),
        [
            "EsServicioPublico",
            "EsRecursosPropios",
            "EsGrupo",
            "EsPrestacionServicios",
            "EsPyme",
            "EstaLiquidado",
            "EsObligacionAmbiental",
            "Es PostConflicto",
        ]
    )
])

one_hot_binarized = Pipeline([
    ("binarizer", binarizer),
    ("one_hot_encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
])

In [161]:
scaler = ColumnTransformer([
    ("scaler", RobustScaler(), ["Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"])
])

In [162]:
passthrough = ColumnTransformer([
    (
        "passthrough",
        "passthrough",
        [

            "Dias Adicionados",
        ]
    )
])

In [163]:
feature_engineering_pipeline = pipe = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    ("categorical", one_hot_encoding),
                    ("categorical_binarized", one_hot_binarized),
                    ("scaled", scaler),
                    ("pass", passthrough)
                ]
            ),
        )
    ]
)

In [164]:
transformed = feature_engineering_pipeline.fit_transform(train_x)
transformed.shape

(48180, 83)

In [165]:
transformed

array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -2.44877760e-01, -1.02671538e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         8.66371520e-01, -9.71583829e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         2.67623841e+01,  5.27654568e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -2.19427120e-01,  9.72672650e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
        -2.85628480e-01, -9.31848641e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         2.15971520e-01,  4.04826595e-01,  0.00000000e+00]])

## Model Trainning

In [166]:
# Get a fresh copy of the pipeline
from sklearn.base import clone

feature_transformer = clone(feature_engineering_pipeline)

features_train_x = feature_transformer.fit_transform(train_x)
features_validate_x = feature_transformer.transform(validate_x)

In [167]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

model = RandomForestClassifier(n_estimators=100)

model.fit(features_train_x, train_y)

RandomForestClassifier()

## Model Validation

In [168]:
from sklearn.metrics import accuracy_score, recall_score

pred_y = model.predict(features_validate_x)

print(accuracy_score(validate_y, pred_y))
print(recall_score(validate_y, pred_y))

0.8947139032438827
0.7928429423459245


## Pipeline Final

In [169]:
final_inference_pipeline = Pipeline([
    ("feature_engineering", clone(feature_engineering_pipeline)),
    ("model", RandomForestClassifier(n_estimators=100))
])

In [170]:
final_training_dataset = pd.concat([train_x, validate_x])
final_training_response = pd.concat([train_y, validate_y])

In [171]:
final_inference_pipeline.fit(final_training_dataset, final_training_response)

Pipeline(steps=[('feature_engineering',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('categorical',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=False),
                                                                                                   ['Orden',
                                                                                                    'Entidad '
                                                                                                    'Centralizada',
                                                                                                    'Modalidad '
                                                                                                    'de '
                                                                                                    'Contratacion',
                                                                                                    'Destino '
                                                                                                    'Gasto',
                                                                                                    'Género '
                                                                                                    'Representante '
                                                                                                    'Legal',
                                                                                                    'Departamento'])]...
                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                 sparse_output=False))])),
                                                                 ('scaled',
                                                                  ColumnTransformer(transformers=[('scaler',
                                                                                                   RobustScaler(),
                                                                                                   ['Valor '
                                                                                                    'del '
                                                                                                    'Contrato',
                                                                                                    'Valor '
                                                                                                    'Facturado',
                                                                                                    'Valor '
                                                                                                    'Pendiente '
                                                                                                    'de '
                                                                                                    'Pago',
                                                                                                    'Saldo '
                                                                                                    'CDP'])])),
                                                                 ('pass',
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                         

## Model testing

In [172]:
test_pred_y = final_inference_pipeline.predict(test_x)

print(accuracy_score(test_pred_y, test_y))
print(recall_score(test_pred_y, test_y))

0.900186799501868
0.8752382969709808


In [173]:

test_x[['proba_NoCerrado','proba_Cerrado']] = final_inference_pipeline.predict_proba(test_x)

In [174]:

test_x.head(10)

,Departamento,Orden,Entidad Centralizada,Modalidad de Contratacion,Destino Gasto,Género Representante Legal,EsServicioPublico,EsRecursosPropios,EsGrupo,EsPrestacionServicios,...,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,Dias Adicionados,Valor del Contrato,Valor Facturado,Valor Pendiente de Pago,Saldo CDP,proba_NoCerrado,proba_Cerrado
2466916,Risaralda,Corporación Autónoma,Centralizada,Contratación directa,Inversión,No Definido,False,True,False,True,...,False,False,False,0,8800000,8800000,0,8800000,0.000000,1.000000
2235081,Meta,Nacional,Descentralizada,Contratación régimen especial,Funcionamiento,Mujer,False,True,False,False,...,False,False,False,0,1648000,0,1648000,3502400,0.160000,0.840000
328631,Antioquia,Territorial,Descentralizada,Contratación directa,Inversión,Mujer,False,True,False,True,...,False,False,False,0,19331130,19331130,0,3617353421,0.978961,0.021039
67633,Antioquia,Territorial,Descentralizada,Contratación directa,Inversión,Mujer,False,False,False,True,...,True,False,False,0,100000000,0,100000000,100000000,0.920000,0.080000
808412,Meta,Nacional,Descentralizada,Mínima cuantía,Funcionamiento,No Definido,False,False,False,False,...,False,False,False,0,32533880,32533880,0,37169524,0.090000,0.910000
835160,Antioquia,Territorial,Descentralizada,Mínima cuantía,Inversión,Hombre,True,False,False,True,...,False,False,False,0,22000000,22000000,22000000,22000000,0.970000,0.030000
685928,Boyacá,Territorial,Centralizada,Contratación directa,Inversión,No Definido,False,False,False,False,...,False,False,False,0,983468704,0,983468704,983468704,0.850000,0.150000
518084,Meta,Territorial,Descentralizada,Contratación directa,Inversión,No Definido,True,False,False,True,...,False,False,False,0,4800000,4800000,0,4800000,0.930000,0.070000
1577961,Antioquia,Territorial,Descentralizada,Contratación régimen especial,Funcionamiento,Hombre,False,False,False,True,...,False,False,False,0,38000000,38000000,0,38000000,0.770000,0.230000
2386441,Cesar,Territorial,Descentralizada,Contratación directa,Funcionamiento,No Definido,True,False,False,True,...,True,False,False,0,22406647,22221468,185179,0,0.960000,0.040000


In [175]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16060 entries, 2466916 to 2150232
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Departamento                16060 non-null  object 
 1   Orden                       16060 non-null  object 
 2   Entidad Centralizada        16060 non-null  object 
 3   Modalidad de Contratacion   16060 non-null  object 
 4   Destino Gasto               16060 non-null  object 
 5   Género Representante Legal  16060 non-null  object 
 6   EsServicioPublico           16060 non-null  bool   
 7   EsRecursosPropios           16060 non-null  bool   
 8   EsGrupo                     16060 non-null  bool   
 9   EsPrestacionServicios       16060 non-null  bool   
 10  EsPyme                      16060 non-null  bool   
 11  EstaLiquidado               16060 non-null  bool   
 12  EsObligacionAmbiental       16060 non-null  bool   
 13  Es PostConflicto            

## Exportando Modelo

In [176]:
from joblib import dump

dump(final_inference_pipeline, "Secop_pipeline.joblib", compress=7)

['Secop_pipeline.joblib']

In [177]:
#import skops.io as sio

#sio.dump(final_inference_pipeline, "Secop_pipeline.sio")

In [178]:
!mv "Secop_pipeline.joblib" "../models/"
#!mv "Secop_pipeline.sio" "../models/"